In [1]:
from pyspark.sql import SparkSession
# Create Spark session
spark = SparkSession.builder.appName("MyApp").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/03 22:24:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# 데이터 로딩 & 샘플 확인
df = spark.read.json("../data/simple-ml")
df.orderBy("value2").show(5)

+-----+----+------+------------------+
|color| lab|value1|            value2|
+-----+----+------+------------------+
|  red|good|    35|14.386294994851129|
| blue| bad|    12|14.386294994851129|
|  red| bad|     2|14.386294994851129|
| blue| bad|     8|14.386294994851129|
|  red| bad|    16|14.386294994851129|
+-----+----+------+------------------+
only showing top 5 rows



데이터셋 (예. 고객 건강 데이터셋)
- color: 범주형 변수(red, blue, ...) -> 서비스 담당자가 작성한 고객의 건강 등급
- lab: 범주형 레이블(good, bad) -> 실제 고객 건강 상태
- value1: 수치형 변수 (웹 서비스내에서의 여러 활동을 숫자로 표현한 척도)
- value2: 수치형 변수

목표: 다른 값으로 이진변수(레이블)를 예측하는 분류 모델을 학습

LIBSVM
- *LIBSVM은 텍스트 기반의 희소 행렬(Sparse Matrix) 형식으로 데이터를 저장.*
- *이 형식은 효율적인 데이터 저장과 빠른 연산을 위해 설계되었으며, 특히 SVM 모델 학습에 적합.*
- 비어 있는 영역이 많고, 실제로 값이 있는 레이블이 제공된다.

### 24.4.1 변환자를 사용해서 피처 엔지니어릴 수행하기

변환자
- 여러 방식으로 현재 칼럼을 조작하는데 사용
- 모델의 입력변수로 사용할 특징을 개발하는 데 초점
- 특징 수를 줄이거나, 특징을 추가하거나, 도출된 특징을 조작하거나, 데이터를 적절히 구성하는데 사용

MLlib 머신러닝 알고리즘의 입력변수
- Double 타입: 레이블용
- Vector[Double] 타입: 특징용

RFormula
- 머신러닝에서 데이터 변환을 지정하기 위한 선언적 언어
- 모델 및 데이터 조작을 위해 R 연산자의 한정된 부분 집합을 지원
  - `~`: 함수에서 타깃과 항을 분리
  - `+`: 연결 기호
  - `-`: 삭제 기호
  - `:`: 상호작용
  - `.`: 타깃/종속변수를 제외한 모든 컬럼

RFormula 클래스를 가져와서 수식을 정의
- 예제. 모든 변수를 사용(.)하고, value1과 color, value2와 color 간의 상호작용을 추가하여 새로운 특징으로 처리


In [4]:
from pyspark.ml.feature import RFormula

supervised = RFormula(formula="lab ~ . + color:value1 + color:value2")

각 컬럼에 대한 가능한 값을 찾아내기 위해 RFormula 변환자를 데이터에 적합

fit 메서드를 호출하면 실제로 데이터를 변형시키는 데 사용할 수 있는 학습된 버전의 변환자를 반환

In [5]:
# DataFrame을 준비
fittedRF = supervised.fit(df)
preparedDF = fittedRF.transform(df)
preparedDF.show()

+-----+----+------+------------------+--------------------+-----+
|color| lab|value1|            value2|            features|label|
+-----+----+------+------------------+--------------------+-----+
|green|good|     1|14.386294994851129|(10,[1,2,3,5,8],[...|  1.0|
| blue| bad|     8|14.386294994851129|(10,[2,3,6,9],[8....|  0.0|
| blue| bad|    12|14.386294994851129|(10,[2,3,6,9],[12...|  0.0|
|green|good|    15| 38.97187133755819|(10,[1,2,3,5,8],[...|  1.0|
|green|good|    12|14.386294994851129|(10,[1,2,3,5,8],[...|  1.0|
|green| bad|    16|14.386294994851129|(10,[1,2,3,5,8],[...|  0.0|
|  red|good|    35|14.386294994851129|(10,[0,2,3,4,7],[...|  1.0|
|  red| bad|     1| 38.97187133755819|(10,[0,2,3,4,7],[...|  0.0|
|  red| bad|     2|14.386294994851129|(10,[0,2,3,4,7],[...|  0.0|
|  red| bad|    16|14.386294994851129|(10,[0,2,3,4,7],[...|  0.0|
|  red|good|    45| 38.97187133755819|(10,[0,2,3,4,7],[...|  1.0|
|green|good|     1|14.386294994851129|(10,[1,2,3,5,8],[...|  1.0|
| blue| ba

features
- 변환을 수행한 결과
- 원시 데이터가 포함

fit(): RFormula가 데이터를 검사하고 RformulaModel이라는 지정된 수식에 따라 데이터를 변환할 객체를 출력

변환자
- 범주형 변수를 Double 타입으로 변환
- color -> 각 범주에 숫자값을 할당
- color + value1, color + value2 -> 상호작용이 반영된 새로운 특징을 만듬
- 모든 특징을 단일 벡터에 넣은 후 입력 데이터를 출력 데이터로 변환하기 위해 해당 객체에 대한 transform을 호출

In [6]:
# 데이터 임의 분할 (학습셋, 테스트셋)
train, test = preparedDF.randomSplit([0.7, 0.3])

### 24.4.2 추정자

예. 로지스틱 회귀 (분류 알고리즘)

분류기 생성
1. 로지스틱회귀 알고리즘 객체화 (기본 설정값)
2. 레이블 칼럼과 특징 컬럼 설정
  - 레이블 이름: label
  - 특징 컬럼: features

In [7]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="label", featuresCol="features")

In [ ]:
# 파라미터 확인
# 스파크를 이용하여 로지스틱 회귀를 구현했을 때 설정된 모든 파라미터의 설명이 출력
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The

In [ ]:
# 알고리즘을 객체화한 다음 학습 데이터에 적합
# 모델을 학습시키기 위한 작업을 수행하고, 학습된 모델을 반환
fittedLR = lr.fit(train)

25/03/03 23:11:39 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [12]:
# transform() 메서드를 사용하여 예측을 수행
fittedLR.transform(train).select("label", "prediction").show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
+-----+----------+
only showing top 20 rows



하이퍼파라미터
- 모델 아키텍처 및 일반화(regularization)와 같은 학습 프로세스에 영향을 주는 설정 매개변수
- 학습을 시작하기 전에 설정

표준화(standarization)
- 평균을 기준으로 관측값들이 얼마나 떨어져 있는지 재표현하는 방법
- 특징의 단위가 2개 이상 다를 때 같은 기준으로 만들어 비교하거나 함께 다를 수 있도록 표준화를 사용
- 예. 수치 데이터 표준화, 카테고리 데이터 표준화, 서수 데이터 표준화

정규화(normalization)
- 데이터의 범위를 바꾸는 방법
  - 데이터 분포의 중심을 0으로 맞추고, 
  - 값의 분포가 특정 범위 안에 들어가도록 조정, 
  - 표준화, 
  - 모든 값을 0에서 1사이의 값으로 재표현

일반화(regularization)
- 모델 과적합을 방지하기 위한 기법
- 모델의 표현식에 추가적인 제약 조건을 걸어 모델이 필요 이상으로 복잡해 지지 않도록 조정해주는 방법
- 복잡한 커브를 많이 가지고 있는 가설함수를 가능한 한 부드럽고 단순하게 만들어주는 기법
- 예. 리지 회귀, 라쏘, 엘라스틱넷, 최소각 회귀

머신러닝
- 표준화 및 정규화 - 스케일링 방법 - 데이터 전처리 과정
- 일반화 - 모델의 일반화 오류를 줄여서 괒거합을 방지하기 위해 사용

### 24.4.3 워크플로를 파이프라인으로 만들기

스파크 Pipeline
- 마지막 단계인 추정자는 조건에 따라 자동으로 조정되며
- 그에 따른 데이터 변환 과정을 설정할 수 있기 때문에
- 튜닝된 모델을 바로 사용할 수 있다. 

![머신러닝 워크플로를 파이프라인으로 만들기](../images/24-4.png)

다른 파이프라인을 생성하기 전에 항상 새로운 모델 객체를 만들어야 한다.

홀드아웃 테스트셋 생성하고 검증셋을 기반으로 하이퍼파라미터를 조정해야한다.

In [13]:
# 학습셋, 훈련셋 분리
train, test = df.randomSplit([0.7, 0.3])

파이프라인의 기본 단계 생성 (변환자, 추정자)

추정자
1. RFomula: 입력 특징을 이해하기 위해 데이터를 분석하고 새로운 특징을 생성하기 위해 형상을 변형
2. LogisticRegression: 모델 생성을 위해 학습을 하는 알고리즘

In [14]:
rForm = RFormula()
lr = LogisticRegression().setLabelCol("label").setFeaturesCol("features")

In [15]:
# 전체 파이프라인에서 단계로 만듬
from pyspark.ml import Pipeline

stages = [rForm, lr]
pipeline = Pipeline().setStages(stages)

### 24.4.4 모델 학습 평가

논리적 파이프라인 -> 모델 학습 -> 평가기, 검증셋, 최적의 모델 선택
- 여러 개의 모델을 학습 (다양한 하이퍼파리미터의 조합을 지정)

RFormula
- 전체 파이프라인에 다양한 하이퍼파라미터를 테스트

In [16]:
from pyspark.ml.tuning import ParamGridBuilder

params = ParamGridBuilder()\
    .addGrid(rForm.formula, [
        "lab ~ . + color:value1",
        "lab ~ . + color:value1 + color:value2"])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .addGrid(lr.regParam, [0.1, 2.0])\
    .build()

예제의 하이퍼파라미터
1. 두 개의 서로 다른 버전의 RFormula
2. 세 개의 서로 다른 옵션의 ElasticNet 파라미터
3. 두 개의 서로 다른 옵션의 일반화 파라미터

12개의 다른 파라미터 조합 -> 12가지 버전의 로지스틱 회귀를 학습

평가 프로세스 지정
- 평가기: 자동으로 여러 모델을 동일한 평가지표를 적용하여 객관적으로 비교
  - BinaryClassificationEvaluator
  - 수신자 조작 특성(receiver operating charateristic) 아래의 areaUderROC

In [17]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()\
    .setMetricName("areaUnderROC")\
    .setRawPredictionCol("prediction")\
    .setLabelCol("label")

파이프라인: 데이터 변환 방법을 지정

모델의 다양한 하이퍼파라미터를 시험하고 성능을 비교하여 모델 적합도를 측정하고 모델을 선택

모델의 과적합을 방지하기 위해 테스트셋 대신 검증셋으로 하이퍼파라미터를 적합시키는 것이 좋다.
- 홀드아웃 테스트셋을 사용할 수 없다.

하이퍼파라미터 튜닝을 자동으로 수행하는 두 가지 옵션 제공
- `TrainValiadationSplit`: 데이터를 두 개의 서로 다른 그룹으로 무작위 임의 분할할 때 사용
- `CrossValidator`: 데이터 집합을 겹치지 않게 임의로 구분된 k개의 폴드로 분할하여 K-겹 교차검증을 수행


In [18]:
from pyspark.ml.tuning import TrainValidationSplit

tvs = TrainValidationSplit()\
    .setTrainRatio(0.75)\
    .setEstimatorParamMaps(params)\
    .setEstimator(pipeline)\
    .setEvaluator(evaluator)

파이프라인 구동
- 검증셋에 대해 모든 버전의 모든 모델이 테스트된다.
- 주어진 모델을 적할시킬 때마다 모델 유형을 출력한다.

In [19]:
# tvs = TrainValidationSplit
tvsFitted = tvs.fit(train)

모델이 테스트셋에서 어떤 성능이 보이는지 평가

In [20]:
evaluator.evaluate(tvsFitted.transform(test))

0.9736842105263158

일부 모델 학습 결과를 요약
- 파이프라인에서 학습 결과를 추출하고
- 적절한 유형으로 전달하여 결과를 출력

In [21]:
from pyspark.ml import PipelineModel
from pyspark.ml.classification import LogisticRegressionModel

# 최적의 파이프라인 모델 로드
trained_pipeline = tvsFitted.bestModel  # PipelineModel

# 파이프라인에서 로지스틱 회귀 모델 가져오기
trained_lr = trained_pipeline.stages[1]  # LogisticRegressionModel

# 모델 요약 정보 가져오기
summary_lr = trained_lr.summary

# 학습 과정에서의 손실 값 출력
print(summary_lr.objectiveHistory)

[0.6810971138759443, 0.48677034522777857, 0.4688225590478846, 0.4604254606220952, 0.4560909637769964, 0.454697425876232, 0.45469352805771984, 0.45469140181985324, 0.45469138765667355, 0.4546913849245738, 0.4546913846346825, 0.45469138461933123, 0.4546913846189616]


### 24.4.5 모델 저장 및 적용

In [ ]:
# 모델을 디스크에 저장
tvsFitted.write().overwrite().save("../tmp/modelLocation")

In [23]:
# 실제 예측을 수행하기 위해 다른 스파크 모델에 모델을 적재할 수 있다.
# 특정 알고리즘에 대한 "모델" 버전을 사용하여 디스크에 저장된 모델을 불러와야 한다.

from pyspark.ml.tuning import TrainValidationSplitModel

# 저장된 모델 로드
model = TrainValidationSplitModel.load("../tmp/modelLocation")

# 모델을 사용하여 예측 수행
predictions = model.transform(test)

# 결과 출력
predictions.show()

/opt/homebrew/Cellar/apache-spark/3.5.4/libexec/python/lib/pyspark.zip/pyspark/daemon.py:154: DeprecationWarning: This process (pid=44352) is multi-threaded, use of fork() may lead to deadlocks in the child.


+-----+----+------+------------------+--------------------+-----+--------------------+--------------------+----------+
|color| lab|value1|            value2|            features|label|       rawPrediction|         probability|prediction|
+-----+----+------+------------------+--------------------+-----+--------------------+--------------------+----------+
| blue| bad|     8|14.386294994851129|(7,[2,3,6],[8.0,1...|  0.0|[2.0340876896385,...|[0.88432986730886...|       0.0|
| blue| bad|     8|14.386294994851129|(7,[2,3,6],[8.0,1...|  0.0|[2.0340876896385,...|[0.88432986730886...|       0.0|
| blue| bad|     8|14.386294994851129|(7,[2,3,6],[8.0,1...|  0.0|[2.0340876896385,...|[0.88432986730886...|       0.0|
| blue| bad|    12|14.386294994851129|(7,[2,3,6],[12.0,...|  0.0|[2.35701370326278...|[0.91349010183322...|       0.0|
| blue| bad|    12|14.386294994851129|(7,[2,3,6],[12.0,...|  0.0|[2.35701370326278...|[0.91349010183322...|       0.0|
| blue| bad|    12|14.386294994851129|(7,[2,3,6]